In [ ]:
import pinyin
from appgeopy import *
from my_packages import *

In [ ]:
# -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- --
def differ_to_ref(series, convert_to="milimeter"):
    """
    Calculate differential values relative to the first measurement in the series.

    Parameters:
    - series (pd.Series): A pandas Series containing the measurements.
    - convert_to (str): Unit to convert the differential values to.
                        Accepts 'milimeter', 'centimeter', or 'meters'.

    Returns:
    - np.ndarray: The differential values converted to the specified unit.

    Raises:
    - ValueError: If 'convert_to' is not 'milimeter', 'centimeter', or 'meters'.
    """
    if convert_to == "milimeter":
        multiplier = 1000
    elif convert_to == "centimeter":
        multiplier = 100
    elif convert_to == "meters":
        multiplier = 1
    else:
        raise ValueError("Invalid 'convert_to' value. Must be 'milimeter', 'centimeter', or 'meters'.")

    # Calculate differential values relative to the first measurement and apply conversion.
    return np.array((series - series.iloc[0]) * multiplier, dtype=np.float16)


# -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- --
def numeric_depth(input_array):
    string_to_num = [eval(ele.split("_")[-1].split("m")[0]) for ele in input_array]
    round_num = np.round(string_to_num, 3)
    return round_num


# -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- --
# Enhanced plotting function
def PlotData_by_Column(ax, df, borehole_depth):
    """
    Plot data from each column of a DataFrame against borehole depth, using smooth interpolation.

    Parameters:
    - ax (matplotlib.axes.Axes): The axis to plot on.
    - df (pd.DataFrame): DataFrame containing columns as dates and values.
    - borehole_depth (np.ndarray): Depth values for the borehole.

    Returns:
    - None
    """
    cmap = plt.get_cmap("turbo")
    norm = Normalize(vmin=df.columns.min().toordinal(), vmax=df.columns.max().toordinal())

    # Generate the date range for normalization of colors
    actual_date_range = pd.date_range(start=df.columns[0], end=df.columns[-1], freq="1D")

    # Vectorize the interpolation process
    for select_col in df.columns:
        arr_label = select_col.strftime("%Y/%m/%d")
        date_as_ordinal = select_col.toordinal()  # Normalize date for color mapping
        color = cmap(norm(date_as_ordinal))

        # Interpolate data using the spline interpolation
        select_array = df[select_col].values
        x_new, y_new = interpolate.spline_interp(x=borehole_depth, y=select_array, factor=100)

        # Plot original data points and the interpolated curve
        ax.plot(-select_array, -borehole_depth, marker="o", linestyle=" ", lw=1.5, color=color)
        ax.plot(-y_new, -x_new, linestyle="-", lw=1.5, color=color)


# -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- --
from mpl_toolkits.axes_grid1.inset_locator import inset_axes


def AddColorBar(fig, ax, df):
    modified_datetime = df.columns.strftime("%Y/%m/%d")
    sm = plt.cm.ScalarMappable(cmap="turbo", norm=plt.Normalize(vmin=0, vmax=1))
    cbaxes = inset_axes(
        ax,
        width="25%",
        height="1.25%",
        loc="lower left",
        bbox_to_anchor=(0.7, 0.05, 0.1, 20),
        bbox_transform=ax.transAxes,
        borderpad=0,
    )
    cbar = fig.colorbar(sm, cax=cbaxes, orientation="vertical")
    cbar.ax.set_yticks([0, 0.2, 0.4, 0.6, 0.8, 1])
    _len = len(modified_datetime)
    datestring_list = [
        modified_datetime[X] for X in [-1, int(_len * 0.8), int(_len * 0.6), int(_len * 0.4), int(_len * 0.2), 0]
    ]
    cbar.ax.set_yticklabels(datestring_list[::-1], fontsize=16)


# -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- --

In [ ]:
mlcw_h5_fpath = r"20241023_MLCW_CRFP_v4.h5"

# Extract existing data and metadata
with h5py.File(mlcw_h5_fpath, "r") as hdf5_file:
    existing_data_dict = gwatertools.h5pytools.hdf5_to_data_dict(hdf5_file)
    existing_metadata_dict = gwatertools.h5pytools.hdf5_to_metadata_dict(hdf5_file)

In [ ]:
stations = sorted(existing_data_dict.keys())
stations[:5]

In [ ]:
# ----------------------------------------------
# Prepare dataframe from HDF5 file
# ----------------------------------------------

# select_station = stations[0]
for select_station in tqdm(stations):
    try:
        # -----------------------------------------------------------------------------
        date_col = existing_data_dict[select_station]["date"]
        date_col = [datetime_handle.bytes_to_datetime(ele) for ele in date_col]
        # -----------------------------------------------------------------------------
        depth_col = existing_data_dict[select_station]["depth"]
        deepest_ring = np.max(depth_col)
        # -----------------------------------------------------------------------------
        ref2base = existing_data_dict[select_station]["values"]["ref2base"]
        # convert to centimeters
        ref2base = ref2base * 100
        # -----------------------------------------------------------------------------
        mlcw_df = pd.DataFrame(data=ref2base, index=depth_col, columns=date_col)
        mlcw_df = mlcw_df.loc[:, "2014":]
        mlcw_df = mlcw_df.sub(mlcw_df.iloc[:, 0], axis=0)
        # -----------------------------------------------------------------------------
        
        # ----------------------------------------------
        # Plot compaction graph
        # ----------------------------------------------
        
        fig = plt.figure(figsize=(8.3 * 2 / 3, 11.7))
        
        ax = fig.add_subplot(111)
        
        PlotData_by_Column(ax, df=mlcw_df, borehole_depth=depth_col)
        
        visualize.configure_axis(
            ax,
            xlabel="Cumulative Compaction (cm)",
            ylabel="Depth (m)",
            title=select_station,
            hide_spines=["bottom", "right"],
            major_tick_length=10,
            minor_tick_length=5,
            tick_direction="in",
            fontsize_base=14,
        )
        
        visualize.configure_ticks(ax=ax, x_major_interval=5, x_minor_interval=1, y_major_interval=50, y_minor_interval=10)
        
        AddColorBar(fig, ax, mlcw_df)
        
        ax.xaxis.set_ticks_position("top")
        ax.xaxis.set_label_position("top")
        
        current_ytick_labels = [label.get_text() for label in ax.get_yticklabels()]
        new_ytick_labels = [ele.replace("−", "") for ele in current_ytick_labels]
        ax.set_yticklabels(new_ytick_labels)
        
        visualize.save_figure(fig, savepath=f"Compaction_Figs\\{select_station.lower()}.png")
        plt.close()
    except Exception as e:
        print(select_station)
        print(e)
        print()
        pass

In [ ]:
select_mlcw = pd.read_table(r"E:\SUBSIDENCE_PROJECT_DATA\地陷資料整理\地陷井\select_MLCW_inCRFP.txt", sep=",")
select_mlcw_CNAME = select_mlcw["檔案名"]
select_mlcw_ENAME = [pinyin.get(ele, format="strip") for ele in select_mlcw_CNAME if len(ele)>1]

source_folder = r"Compaction_Figs"
destination_folder = r"Compaction_Figs\from_2014_to_latest"
for name in select_mlcw_ENAME:
    filename = f"{name}.png"
    source_fpath = os.path.join(source_folder, filename)
    destination_fpath = os.path.join(destination_folder, filename)
    shutil.move(src=source_fpath, dst=destination_fpath)